In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import scipy.stats as ss
import statsmodels.api as sm
import pickle

/home/harshit/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [25]:
lis = pd.DataFrame(os.listdir("/home/harshit/Project/complete_models/5years/"))
coeff = pd.DataFrame(columns = ["Symbol","const","SMB %", "HML %","Rm-Rf %", "high-low"] )
#Making a dataframe of all coffecients
for i in lis[0]:
    data = open("/home/harshit/Project/complete_models/5years/"+i,"rb")
    symbol = i.split('.')[0]
    model = pickle.load(data)
    frame = pd.DataFrame(model.params).T
    frame.insert(0,"Symbol",symbol)
    coeff = coeff.append(frame,ignore_index=True,sort=False)
    coeff= coeff[0:1200]
intercept =pd.DataFrame(coeff['const'])

In [40]:
all_factors= pd.read_csv("/home/harshit/Project/allfactors.csv")

In [30]:
#Calculating residuals
model_path = "/home/harshit/Project/complete_models/5years/"
return_path = "/home/harshit/Project/risk_free_ret/all_ret_by_company/"
mod = os.listdir(model_path)
ret = os.listdir(return_path)
residuals = pd.DataFrame(all_factors["Date"])
returns = pd.DataFrame(all_factors["Date"])
for i in coeff["Symbol"]:
    data = open("/home/harshit/Project/complete_models/5years/"+i+".sav","rb") # loading model
    model = pickle.load(data)
    equity = pd.read_csv(return_path+i+".csv")                       # loading daily Return data
    all_data = pd.merge(all_factors,equity,on="Date")
    all_data.rename(columns={'Return':i}, inplace=True)
    X= all_data[["SMB %","HML %","WML %","Rm-Rf %","high-low"]] #Making X variable for entropy included model
    X= sm.add_constant(X)
    y = pd.DataFrame(all_data[i])
    estimated = pd.DataFrame(model.predict(X),columns=[i])
    return1 = y
    residual = y - estimated
    return1 = pd.merge(all_data[["Date"]],return1,left_index=True,right_index=True)
    residual= pd.merge(all_data[["Date"]],residual,left_index=True, right_index=True)
    residuals= pd.merge_asof(residuals,residual,on="Date")
    returns = pd.merge_asof(returns,return1,on="Date")
    residual.to_csv("/home/harshit/Project/residual/"+i+".csv",index=False)
#residuals.to_csv("/home/harshit/Project/residual/combined.csv",index=False)

    
#residuals.reset_index(inplace=True)


In [38]:
residual_matrix = residuals.drop(["Date"],axis=1)

(1232, 1200)

In [70]:
T=len(residual_matrix.index)      #Time set
N = len(residual_matrix.columns) #No. of firms
L = len(all_factors.columns)         #no. of factors  
resi_cov=residual_matrix.cov()  #unbiased estimate of the covariance matrix of residuals

22

In [43]:
factor_path = "/home/harshit/Project/allfactors.csv"
factors = pd.read_csv(factor_path)
factors = factors[['SMB %','HML %','WML %','Rm-Rf %','high-low']]    
mean_factors = pd.DataFrame(factors.mean(),columns = ["Mean"])  # Matrix of mean of factors
factors_cov=factors.cov()                    #unbiasesd covariance of Factors

In [83]:
resi_cov_inv= pd.DataFrame(np.linalg.pinv(resi_cov)) # inverse of residual matrix
factors_cov_inv = pd.DataFrame(np.linalg.pinv(factors_cov))
stat = (T/N)*(T-L-N/T-L-1)*(intercept.transpose().dot(resi_cov_inv).dot(intercept)) # numerator of stat
stat = stat/(1+mean_factors.values.transpose().dot(factors_cov_inv).dot(mean_factors))
stat
ss.f.pdf(stat.iat[0,0],N,(T-N-L))      #p-value of f distribution

1.9311863519578344e-58